# Install Libraries

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann
!pip install tflite-support

# Import Libraries

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

import os
import pprint
import tempfile

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pathlib
from google.colab import files



# Read CSV Dataset

In [ ]:
users = pd.read_csv('https://raw.githubusercontent.com/mutakin-san/valo.ai/main/machine-learning/dataset/user_valo_mod_str.csv')
users.head()

,user_id,name,age,gender,vac_1,vac_2,vac_booster
0,1,Bakda Salahudin,18,1,Sinopharm,Sinopharm,Janssen
1,2,Eja Widodo,36,1,Sinopharm,Sinopharm,Janssen
2,3,Gamanto Prasetya,39,1,Sinopharm,Sinopharm,Janssen
3,4,Vera Puspita,47,0,Sinopharm,Sinopharm,Janssen
4,5,Elvina Pratiwi,46,0,Sinopharm,Sinopharm,Janssen


# Input Shape

In [ ]:
shape = users.shape
shape

(102400, 7)

# Data Preparation

In [ ]:
grouped_data_dict = users.groupby(['user_id', 'vac_booster'])['vac_1', 'vac_2'].sum().reset_index()

grouped_data_dict = {name: np.array(value) for name, value in grouped_data_dict.items()}
grouped_data = tf.data.Dataset.from_tensor_slices(grouped_data_dict)

booster_dict = users[['vac_booster']].drop_duplicates()
booster_dict = {name: np.array(value) for name, value in booster_dict.items()}
booster = tf.data.Dataset.from_tensor_slices(booster_dict)

grouped_data = grouped_data.map(lambda x: {
    'user_id' : str(x['user_id']), 
    'vac_booster' : x['vac_booster'], 
    'vac_1' : x['vac_1'],
    'vac_2' : x['vac_2'],

})

booster = booster.map(lambda x: x['vac_booster'])

In [ ]:
l = list(grouped_data)
l[0]

In [ ]:
booster

In [ ]:
unique_boosters = np.unique(np.concatenate(list(booster.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(grouped_data.batch(1_000).map(lambda x: x["vac_1"]))))
unique_user_ids2 = np.unique(np.concatenate(list(grouped_data.batch(1_000).map(lambda x: x["vac_2"]))))

# Shuffle the Data Randomly and Split for Train and Test

In [ ]:
tf.random.set_seed(42)
shuffled = grouped_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(60_000)
test = shuffled.skip(60_000).take(40_000)

# Build Retrieval Model for Vaccine

In [ ]:
class VaccineModel(tfrs.Model):
  def __init__(self, user_model, booster_model):
    super().__init__()
    booster_model = tf.keras.Sequential([
                                      tf.keras.layers.experimental.preprocessing.StringLookup(
                                          vocabulary=unique_boosters, mask_token=None),
                                      tf.keras.layers.Embedding(len(unique_boosters) + 1, embedding_dimension)
    ])
    self.booster_model: tf.keras.Model = booster_model

    user_model = tf.keras.Sequential([
                                      tf.keras.layers.experimental.preprocessing.StringLookup(
                                          vocabulary=unique_user_ids, mask_token=None),
                                      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.Model = user_model

    metrics = tfrs.metrics.FactorizedTopK(
      candidates=booster.batch(512).map(booster_model)
    )

    task = tfrs.tasks.Retrieval(metrics=metrics)
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features['vac_1'], features['vac_2'])
    positive_vac_embeddings = self.booster_model(features['vac_booster'])
    return self.task(user_embeddings, positive_vac_embeddings)

# Fitting and Evaluating

In [ ]:
embedding_dimension = 32

booster_model = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                      vocabulary=unique_boosters, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_boosters) + 1, embedding_dimension)
])

user_model = tf.keras.Sequential([
                                  tf.keras.layers.experimental.preprocessing.StringLookup(
                                      vocabulary=unique_user_ids, mask_token=None),
                                  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

model = VaccineModel(user_model, booster_model)

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

history = model.fit(cached_train, epochs=10)
model.evaluate(cached_test, return_dict=True)

# Plot the Accuracy and Epochs

In [ ]:
epochs = [i for i in range(10)]

plt.plot(epochs, history.history["factorized_top_k/top_5_categorical_accuracy"], label="accuracy")
plt.title("Accuracy vs epoch")
plt.xlabel("epoch")
plt.ylabel("Top-100 accuracy");
plt.legend()

# Predict Top Recommended Vaccines

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=3)
index.index_from_dataset(
    booster.batch(128).map(lambda title: (title, model.booster_model(title)))
)

In [ ]:
_, vac = index(tf.constant(["AZ", "AZ"]))
print(f"Recommendations: {vac[0]}")

Recommendations: [b'Pfizer' b'AZ' b'Moderna']


# Save the Model and Load Saved Model

In [ ]:
tf.saved_model.save(index, "export")

In [ ]:
# Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load("export")

# Pass a user id in, get top predicted vaccines
scores, vaccines = loaded(["AZ", "AZ"])

print(f"Recommendations: {vaccines[0]}")

In [ ]:
print(loaded.signatures['serving_default'].inputs)

In [ ]:
print(f"Recommendations: {scores[0]}")

# Convert Saved Model to TFLite Model

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("export")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

# Using Interpreter to Get Predicted Vaccine from TFLite Model

In [ ]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

# test the model
input_shape = input_details[0]['shape']
print(input_shape)

interpreter.set_tensor(input_details[0]['index'], np.array(["Janssen"]))
# interpreter.set_tensor(input_details[0]['index'], np.array(["Pfizer"]))

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[1]['index'])
print(output_data[0])